In [ ]:
import requests
import json
import time

In [ ]:
# fetching games list from Free2game
def get_free2game_games():
    url = ("https://www.freetogame.com/api/games")

    response = requests.get(url)

    if not response.status_code == 200:
        print("ERROR, request not OK")
    
    data = response.json()
    return data

In [ ]:
data = get_free2game_games()

In [ ]:
# storing the api data in local json file
with open("free2game_games.json", 'w') as my_file:
    data_string = json.dumps(data)
    print(data_string, file=my_file)

In [ ]:
# fetching games list from steam api

In [ ]:
def get_steam_games():
    url = ("https://api.steampowered.com/ISteamApps/GetAppList/v2/")

    response = requests.get(url)

    if not response.status_code == 200:
        print("ERROR, request not OK")
    
    data = response.json()
    return data

In [ ]:
steam_data = get_steam_games()

In [ ]:
ids = [item.get("appid") for item in steam_data['applist']['apps']]

In [ ]:
len(ids)

In [ ]:
first_batch = ids[0:82255]

In [ ]:
first_batch_update = ids[1741:10000]

In [ ]:
def get_steam_games_info(gameId):
    url = ("https://store.steampowered.com/api/appdetails?appids="+f"{gameId}")

    response = requests.get(url)

    if not response.status_code == 200:
        print("ERROR, request not OK",gameId)
    else:
        data = response.json()
        return data

In [ ]:
with open("steam_games_first_batch.json", 'w') as f:
    for appid in first_batch:
        game_info = get_steam_games_info(appid)
        data_string= json.dumps(game_info)
        print(data_string,file=f)
        time.sleep(3)

In [ ]:
### data cleaning

In [ ]:
with open("free2game_games.json", 'r') as f2g:
    f2g_data = f2g.read()

In [ ]:
f2g_input = json.loads(f2g_data)

In [ ]:
genre_counts = {}  # Dictionary to store genre counts

# Iterate over the list of IDs
for game in f2g_input:
    genre = game.get("genre")
    if genre:
        if genre in genre_counts:
            genre_counts[genre] += 1
        else:
            genre_counts[genre] = 1

# Print the genre counts
for genre, count in genre_counts.items():
    print(f"{genre}: {count}")

In [ ]:
from datetime import datetime

In [ ]:
year_counts = {}  # Dictionary to store year counts

# Iterate over the list of IDs
for game in f2g_input:
    release_date_str = game.get("release_date")
    if release_date_str:
        
        release_date_parts = release_date_str.split('-')
        if (release_date_parts[2] == '00'):
            # Replace the day value with '01'
            release_date_parts[2] = '01'
            release_date_str = '-'.join(release_date_parts)
        
        release_date = datetime.strptime(release_date_str, "%Y-%m-%d")
        year = release_date.year

        if year in year_counts:
            year_counts[year] += 1
        else:
            year_counts[year] = 1
            
sorted_dates = sorted(year_counts.items())
# Print the counts of games released each year
for year, count in sorted_dates:
    print(f"{year}: {count}")

In [ ]:
with open("f2g_genre_games.tsv", "w") as f:
    print("Genre\tNum_of_Games", file=f)   
    
    for gen in genre_counts.keys():
        genre_num = genre_counts[gen]
        print(gen, "\t", genre_num, file=f)

In [ ]:
with open("f2g_year_games.tsv", "w") as f:
    print("Year\tNum_of_Games", file=f)   
    
    for year in year_counts.keys():
        year_num = year_counts[year]
        print(year, "\t", year_num, file=f)

In [ ]:
#Steam Data Processing

In [ ]:
with open("steam_games_first_batch.json", 'r') as steam:
    steam_json = steam.read()

In [ ]:
data = [json.loads(line)
        for line in open('steam_games_first_batch.json', 'r', encoding='utf-8')]

In [ ]:
combined_dict = {}
for item in data:
    combined_dict.update(item)

In [ ]:
# getting all the games which has information : success = true
success_data = {}
for key, value in combined_dict.items():
    success = value.get("success")
    if success:
        success_data[key] = value

In [ ]:
# total games for which api information is available
len(success_data)

In [ ]:
## getting types of games in steam
game_types_counts = {}

for app_id, game_info in success_data.items():
    game_type = game_info['data'].get('type', 'Unknown')
    if game_type in game_types_counts:
        game_types_counts[game_type] += 1
    else:
        game_types_counts[game_type] = 1

print("Game Types and Their Counts:")
for game_type, count in game_types_counts.items():
    print(game_type, ":", count)

In [ ]:
# describing all the genres available in steam
genre_descriptions = {}

for app_id, game_info in success_data.items():
    genres = game_info['data'].get('genres', [])
    for genre in genres:
        genre_id = genre.get('id')
        genre_description = genre.get('description')
        if genre_id and genre_description:
            genre_descriptions[genre_id] = genre_description

print("Genre Descriptions:")
for genre_id, description in genre_descriptions.items():
    print(f"Genre ID: {genre_id}, Description: {description}")

In [ ]:
# getting counts of each genre
genre_counts = {}

for app_id, game_info in success_data.items():
    genres = game_info['data'].get('genres', [])
    for genre in genres:
        genre_id = genre.get('id')
        genre_description = genre.get('description')
        if genre_id and genre_description:
            if genre_id in genre_counts:
                genre_counts[genre_id] += 1
            else:
                genre_counts[genre_id] = 1

print("Genre Counts:")
for genre_id, count in genre_counts.items():
    print(f"Genre ID: {genre_id}, Count: {count}")

In [ ]:
# Dictionary to store description and count
description_dict = {}  

for app_id, description in genre_descriptions.items():
    if app_id in genre_counts:
        count = genre_counts[app_id]
        description_dict[description] = count

description_dict

In [ ]:
from collections import defaultdict
import operator

developer_counts = defaultdict(int)

for app_id, game_info in success_data.items():
    developers = game_info['data'].get('developers', [])
    for developer in developers:
        developer_counts[developer] += 1

top_10_developers = sorted(developer_counts.items(), key=operator.itemgetter(1), reverse=True)[:10]

print("Top 10 Developers:")
for developer, count in top_10_developers:
    print(f"Developer: {developer}, Count: {count}")

In [ ]:
# getting the count of free games available on steam
free_games_count = 0

for app_id, game_info in success_data.items():
    is_free = game_info['data'].get('is_free', False)
    if is_free is True:
        free_games_count += 1

print("Count of Free Games:", free_games_count)

In [ ]:
with open("steam_gametypes.tsv", "w") as f:
    print("Game_type\tNum_of_Games", file=f)   
    
    for gtype in game_types_counts.keys():
        gen_type_num = game_types_counts[gtype]
        print(gtype, "\t", gen_type_num, file=f)

In [ ]:
with open("steam_gamegenre.tsv", "w") as f:
    print("Game_genre\tNum_of_Games", file=f)   
    
    for genre in description_dict.keys():
        genre_number = description_dict[genre]
        print(genre, "\t", genre_number, file=f)